# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
import create_tables

In [2]:
# Create database (sparkifydb) and tables for this project
create_tables.main()

In [3]:
# Create connection and Get a cursor that we will use to execute queries
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
# Use automactic commit 
conn.set_session(autocommit=True)

In [5]:
# Define get_files function to get a JSON files list in defined filepath
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
## DONE: Use the get_files function to get a list of all song JSON files in data/song_data

song_files = []
filepath = get_files('data/song_data')

In [7]:
## DONE: Select the first song in this list

f_song=pd.read_json(filepath[0],orient='index').T
f_song

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,None,California - LA,None,Casual,218.932,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [8]:
## DONE: Read the song file and view the data

for i in filepath:
    song_files.append(pd.read_json(i,orient='records',lines=True))
    
df=pd.concat(song_files,ignore_index=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007



## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
## Select columns for song ID, title, artist ID, year, and duration
#  ['num_songs', 'artist_id', 'artist_latitude','artist_longitude','artist_location','artist_name','song_id','title','duration','year']
columns_songs = ['song_id','title','artist_id','year','duration']
df_songs = df[columns_songs]
df_songs

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
1,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
2,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.47546
3,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
4,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
...,...,...,...,...,...
66,SOINLJW12A8C13314C,City Slickers,AR8IEZO1187B99055E,2008,149.86404
67,SOGDBUF12A8C140FAA,Intro,AR558FS1187FB45658,2003,75.67628
68,SORRZGD12A6310DBC3,Harajuku Girls,ARVBRGZ1187FB4675A,2004,290.55955
69,SODAUVL12A8C13D184,Prognosis,ARWB3G61187FB49404,2000,363.85914


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
# Return a Numpy representation of the DataFrame

song_data=df_songs.values
song_data

array([['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1',
        0, 218.93179],
       ['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969,
        148.03546],
       ['SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0,
        177.47546],
       ['SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C',
        1982, 233.40363],
       ['SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099',
        2007, 209.60608],
       ['SOYMRWW12A6D4FAB14',
        'The Moon And I (Ordinary Day Album Version)',
        'ARKFYS91187B98E58F', 0, 267.7024],
       ['SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)',
        'ARD0S291187B9B7BF5', 0, 114.78159],
       ['SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0,
        189.57016],
       ['SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984,
        269.81832],
       ['SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)',
        'ARNTLGG11E2835DDB9', 0, 266.39628],
   

In [11]:
#  Insert records into the songs table

for s_data in song_data:
    cur.execute(song_table_insert, s_data)
    conn.commit()

In [12]:
# Verify if rows were inserted into song table

try:
    cur.execute('select * from songs')
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()   

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, Decimal('218.93179'))
('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, Decimal('148.03546'))
('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, Decimal('177.47546'))
('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, Decimal('233.40363'))
('SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, Decimal('209.60608'))
('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', 0, Decimal('267.7024'))
('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, Decimal('114.78159'))
('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, Decimal('189.57016'))
('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, Decimal('269.81832'))
('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, Decimal('266.39628'))
('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
# Select columns for song ID, title, artist ID, year, and duration
#  ['num_songs', 'artist_id', 'artist_latitude','artist_longitude','artist_location','artist_name','song_id','title','duration','year']
columns_artists = ['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']
df_artists = df[columns_artists]
df_artists

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
1,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
2,ARKRRTF1187B9984DA,Sonora Santanera,,NaN,NaN
3,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
4,ARXR32B1187FB57099,Gob,,NaN,NaN
...,...,...,...,...,...
66,AR8IEZO1187B99055E,Marc Shaiman,,NaN,NaN
67,AR558FS1187FB45658,40 Grit,,NaN,NaN
68,ARVBRGZ1187FB4675A,Gwen Stefani,,NaN,NaN
69,ARWB3G61187FB49404,Steve Morse,"Hamilton, Ohio",NaN,NaN


In [14]:
# Return a Numpy representation of the DataFrame
artist_data = df_artists.values
artist_data

array([['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan],
       ['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968,
        -90.04892],
       ['ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan],
       ['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan],
       ['ARXR32B1187FB57099', 'Gob', '', nan, nan],
       ['ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan],
       ['ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan],
       ['AR10USD1187B99F3F1', 'Tweeterfriendly Music',
        'Burlington, Ontario, Canada', nan, nan],
       ['AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan],
       ['ARNTLGG11E2835DDB9', 'Clp', '', nan, nan],
       ['ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962,
        -80.01955],
       ['ARC43071187B990240', 'Wayne Watson', 'Wisner, LA', nan, nan],
       ['ARL7K851187B99ACD2', 'Andy Andy', '', nan, nan],
       ['ARHHO3O1187B989413', 'Bob Azzam', '', nan, nan],
       ['ARIK43K1

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
#  Insert records into the artists table
for a_data in artist_data:
    cur.execute(artist_table_insert, a_data)
    conn.commit()

In [16]:
# Verify if rows were inserted into artists table

try:
    cur.execute('select * from songs')
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()  

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, Decimal('218.93179'))
('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, Decimal('148.03546'))
('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, Decimal('177.47546'))
('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, Decimal('233.40363'))
('SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, Decimal('209.60608'))
('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', 0, Decimal('267.7024'))
('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, Decimal('114.78159'))
('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, Decimal('189.57016'))
('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, Decimal('269.81832'))
('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, Decimal('266.39628'))
('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = 

In [ ]:
filepath = 

In [ ]:
df = 
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.